Challenge 1 : BigQuery - Feature Engineering

In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import Conflict


In [ ]:
client = bigquery.Client()

PROJECT_ID = client.project
DATASET_ID = "fraud_dataset"
TABLE_ID = "fraud_data_new"
LOCATION = "us-central1"

dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"
table_ref = f"{dataset_ref}.{TABLE_ID}"

dataset = bigquery.Dataset(dataset_ref)
dataset.location = LOCATION

try:
    client.create_dataset(dataset, timeout=30)
    print(f"Dataset {DATASET_ID} created.")
except Conflict:
    print(f"Dataset {DATASET_ID} already exists.")



Dataset fraud_dataset already exists.


In [ ]:

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    write_disposition="WRITE_TRUNCATE"
)

file_uri = "gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv"

load_job = client.load_table_from_uri(
    file_uri, table_ref, job_config=job_config
)

load_job.result()
print(f"Successfully loaded {load_job.output_rows} rows into {table_ref}.")

Successfully loaded 50000 rows into qwiklabs-gcp-01-5f9c2ba2a4cd.fraud_dataset.fraud_data_new.


In [ ]:
%%bigquery fraud_df
SELECT * FROM fraud_dataset.fraud_data_new

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
import pandas as pd

In [ ]:
fraud_df.head(5)

,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Device_Type,Application_Date,Fraudulent
0,217,65,Unemployed,28984,4,5872,False,NaT,False,1,156.133.45.45,Mobile,2024-08-18,0
1,226,54,Self-Employed,0,1,6631,False,NaT,False,1,245.13.80.245,Tablet,2024-05-11,0
2,240,26,Self-Employed,64477,5,8612,False,NaT,True,1,213.103.170.95,Mobile,2024-08-14,0
3,252,28,Unemployed,28576,4,2951,False,NaT,True,1,234.179.149.207,Desktop,2024-06-12,0
4,266,43,Employed,44930,5,2324,False,NaT,False,1,66.109.96.227,Mobile,2024-08-16,0


In [ ]:
fraud_df.describe()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Application_Frequency_Last_Year,Fraudulent
count,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0
mean,25200.5,41.572,21189.00928,2.50302,5238.93376,5.50224,0.06474
std,14433.901067,13.843572,24972.660972,1.704948,2732.937807,2.870094,0.246069
min,201.0,18.0,0.0,0.0,500.0,1.0,0.0
25%,12700.75,30.0,0.0,1.0,2881.0,3.0,0.0
50%,25200.5,42.0,10060.0,2.0,5239.0,5.0,0.0
75%,37700.25,54.0,42135.5,4.0,7585.25,8.0,0.0
max,50200.0,65.0,75000.0,5.0,10000.0,10.0,1.0


In [ ]:
fraud_df.dtypes

,0
Applicant_ID,Int64
Age,Int64
Employment_Status,object
Income,Int64
Number_of_Dependents,Int64
Amount_Requested,Int64
Previous_Assistance_Received,boolean
Previous_Assistance_Date,dbdate
Supporting_Doc_Verified,boolean
Application_Frequency_Last_Year,Int64


In [ ]:
fraud_df_new = pd.DataFrame()

In [ ]:
fraud_df_new = fraud_df[['Applicant_ID','Employment_Status', 'Device_Type', 'Age', 'Income','Amount_Requested','Previous_Assistance_Received', 'Supporting_Doc_Verified', 'Previous_Assistance_Date', 'Application_Date']]

In [ ]:
fraud_df_new.head(5)

,Applicant_ID,Employment_Status,Device_Type,Age,Income,Amount_Requested,Previous_Assistance_Received,Supporting_Doc_Verified,Previous_Assistance_Date,Application_Date
0,217,Unemployed,Mobile,65,28984,5872,False,False,NaT,2024-08-18
1,226,Self-Employed,Tablet,54,0,6631,False,False,NaT,2024-05-11
2,240,Self-Employed,Mobile,26,64477,8612,False,True,NaT,2024-08-14
3,252,Unemployed,Desktop,28,28576,2951,False,True,NaT,2024-06-12
4,266,Employed,Mobile,43,44930,2324,False,False,NaT,2024-08-16


In [ ]:
bins = [18, 24, 34, 44, 54, 64, 74]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-74']

fraud_df_new['Age_Bins'] = pd.cut(fraud_df_new['Age'], bins=bins, labels=labels, include_lowest=True)


/tmp/ipython-input-508363042.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fraud_df_new['Age_Bins'] = pd.cut(fraud_df_new['Age'], bins=bins, labels=labels, include_lowest=True)


In [ ]:
fraud_df_new.head(5)

,Applicant_ID,Employment_Status,Device_Type,Age,Income,Amount_Requested,Previous_Assistance_Received,Supporting_Doc_Verified,Previous_Assistance_Date,Application_Date,Age_Bins
0,217,Unemployed,Mobile,65,28984,5872,False,False,NaT,2024-08-18,65-74
1,226,Self-Employed,Tablet,54,0,6631,False,False,NaT,2024-05-11,45-54
2,240,Self-Employed,Mobile,26,64477,8612,False,True,NaT,2024-08-14,25-34
3,252,Unemployed,Desktop,28,28576,2951,False,True,NaT,2024-06-12,25-34
4,266,Employed,Mobile,43,44930,2324,False,False,NaT,2024-08-16,35-44


In [ ]:
fraud_df_new = pd.get_dummies(fraud_df_new, columns=['Employment_Status', 'Device_Type', 'Age_Bins'], prefix='enc', dtype=int)

In [ ]:
fraud_df_new.head()

,Applicant_ID,Age,Income,Amount_Requested,Previous_Assistance_Received,Supporting_Doc_Verified,Previous_Assistance_Date,Application_Date,enc_Employed,enc_Self-Employed,enc_Unemployed,enc_Desktop,enc_Mobile,enc_Tablet,enc_18-24,enc_25-34,enc_35-44,enc_45-54,enc_55-64,enc_65-74
0,217,65,28984,5872,False,False,NaT,2024-08-18,0,0,1,0,1,0,0,0,0,0,0,1
1,226,54,0,6631,False,False,NaT,2024-05-11,0,1,0,0,0,1,0,0,0,1,0,0
2,240,26,64477,8612,False,True,NaT,2024-08-14,0,1,0,0,1,0,0,1,0,0,0,0
3,252,28,28576,2951,False,True,NaT,2024-06-12,0,0,1,1,0,0,0,1,0,0,0,0
4,266,43,44930,2324,False,False,NaT,2024-08-16,1,0,0,0,1,0,0,0,1,0,0,0


In [ ]:
fraud_df_new['Income-to-Amount-Requested'] = fraud_df_new['Income'] / fraud_df_new['Amount_Requested']

In [ ]:
fraud_df_new['Application_Date'] = pd.to_datetime(fraud_df_new['Application_Date'])

fraud_df_new['Previous_Assistance_Date'] = pd.to_datetime(fraud_df_new['Previous_Assistance_Date'])

fraud_df_new['Time_Since_Previous_Assistance'] = (fraud_df_new['Application_Date'] - fraud_df_new['Previous_Assistance_Date']).dt.days

fraud_df_new['Time_Since_Previous_Assistance'] = fraud_df_new['Time_Since_Previous_Assistance'].fillna(-1).astype(int)

In [ ]:

fraud_df_new['Previous_Assistance_Received'] = fraud_df_new['Previous_Assistance_Received'].astype(int)
fraud_df_new['Supporting_Doc_Verified'] = fraud_df_new['Supporting_Doc_Verified'].astype(int)

In [ ]:

fraud_df_new.head(5)

,Applicant_ID,Age,Income,Amount_Requested,Previous_Assistance_Received,Supporting_Doc_Verified,Previous_Assistance_Date,Application_Date,enc_Employed,enc_Self-Employed,...,enc_Mobile,enc_Tablet,enc_18-24,enc_25-34,enc_35-44,enc_45-54,enc_55-64,enc_65-74,Income-to-Amount-Requested,Time_Since_Previous_Assistance
0,217,65,28984,5872,0,0,NaT,2024-08-18,0,0,...,1,0,0,0,0,0,0,1,4.935967,-1
1,226,54,0,6631,0,0,NaT,2024-05-11,0,1,...,0,1,0,0,0,1,0,0,0.0,-1
2,240,26,64477,8612,0,1,NaT,2024-08-14,0,1,...,1,0,0,1,0,0,0,0,7.486879,-1
3,252,28,28576,2951,0,1,NaT,2024-06-12,0,0,...,0,0,0,1,0,0,0,0,9.683497,-1
4,266,43,44930,2324,0,0,NaT,2024-08-16,1,0,...,1,0,0,0,1,0,0,0,19.333046,-1


In [ ]:
fraud_df_new.describe()

,Applicant_ID,Age,Income,Amount_Requested,Previous_Assistance_Received,Supporting_Doc_Verified,Previous_Assistance_Date,Application_Date,enc_Employed,enc_Self-Employed,...,enc_Mobile,enc_Tablet,enc_18-24,enc_25-34,enc_35-44,enc_45-54,enc_55-64,enc_65-74,Income-to-Amount-Requested,Time_Since_Previous_Assistance
count,50000.0,50000.0,50000.0,50000.0,50000.000000,50000.000000,24854,50000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.00000
mean,25200.5,41.572,21189.00928,5238.93376,0.497080,0.502920,2023-10-22 07:40:19.216222720,2024-07-02 07:33:25.632000,0.334680,0.333640,...,0.334660,0.333040,0.14622,0.205380,0.205440,0.213780,0.208760,0.020420,6.664208,125.71898
min,201.0,18.0,0.0,500.0,0.000000,0.000000,2022-08-20 00:00:00,2024-01-01 00:00:00,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-1.00000
25%,12700.75,30.0,0.0,2881.0,0.000000,0.000000,2023-06-16 00:00:00,2024-04-02 00:00:00,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-1.00000
50%,25200.5,42.0,10060.0,5239.0,0.000000,1.000000,2023-10-22 00:00:00,2024-07-02 00:00:00,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.095277,-1.00000
75%,37700.25,54.0,42135.5,7585.25,1.000000,1.000000,2024-02-28 00:00:00,2024-10-02 00:00:00,1.000000,1.000000,...,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,8.112387,252.00000
max,50200.0,65.0,75000.0,10000.0,1.000000,1.000000,2024-12-21 00:00:00,2024-12-31 00:00:00,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,146.996016,500.00000
std,14433.901067,13.843572,24972.660972,2732.937807,0.499996,0.499996,NaN,NaN,0.471883,0.471518,...,0.471876,0.471305,0.35333,0.403983,0.404027,0.409977,0.406427,0.141433,12.936536,162.43499


In [ ]:

DEST_TABLE_ID = 'fraud_training_data'
dest_table_ref =  f"{dataset_ref}.{DEST_TABLE_ID}"


job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    fraud_df_new, dest_table_ref, job_config=job_config
)

# Wait for the job to complete
job.result()

print(f"Successfully saved {len(fraud_df_new)} rows to {dest_table_ref}")

Successfully saved 50000 rows to qwiklabs-gcp-01-5f9c2ba2a4cd.fraud_dataset.fraud_training_data
